In [1]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "tamiti1610001/bert-finetuned-squad"
question_answerer = pipeline("question-answering", model=model_checkpoint)

context = """

প্রধানমন্ত্রীর বেসরকারি শিল্প ও বিনিয়োগ বিষয়ক উপদেষ্টা সালমান এফ রহমানের সঙ্গে বৈঠক করেছেন ঢাকায় নিযুক্ত যুক্তরাষ্ট্রের রাষ্ট্রদূত পিটার হাস। বুধবার (৯ আগস্ট) বিকাল সাড়ে ৪টা থেকে ৫টা পর্যন্ত গুলশানে সালমান এফ রহমানের বাসায় এ বৈঠক অনুষ্ঠিত হয় বলে একাধিক সূত্র বাংলা ট্রিবিউনকে নিশ্চিত করেছে। তবে বৈঠকের আলোচনার বিষয়ে কিছু জানা যায়নি।

"""
question = "যুক্তরাষ্ট্রের রাষ্ট্রদূত কে?"
question_answerer(question=question, context=context)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

{'score': 0.9946432709693909, 'start': 133, 'end': 142, 'answer': 'পিটার হাস'}

In [2]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [4]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

In [5]:
import torch

inputs = tokenizer(question, context, add_special_tokens=True, return_tensors="pt")
#encoded_inputs = tokenizer.encode(inputs, return_tensors="pt")
input_ids = inputs["input_ids"][0]
outputs = model(**inputs)
answer_start_scores = outputs.start_logits
answer_end_scores = outputs.end_logits
# Get the most likely beginning of answer with the argmax of the score
answer_start = torch.argmax(answer_start_scores)
# Get the most likely end of answer with the argmax of the score
answer_end = torch.argmax(answer_end_scores) + 1
answer = tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end])
)
print(f"Question: {question}")
print(f"Answer: {answer}")

Question: যুক্তরাষ্ট্রের রাষ্ট্রদূত কে?
Answer: পিটার হাস


In [6]:
#%debug

In [7]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.9 MB/s eta 0:00:00


In [8]:
from datasets import load_dataset
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)



In [9]:
raw_datasets = load_dataset("glue", "mnli")

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/5 [00:00<?, ?it/s]

In [10]:
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


In [11]:
def preprocess_function(examples):
    return tokenizer(examples["premise"], examples["hypothesis"], truncation=True)


In [12]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)



  0%|          | 0/393 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [13]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels = 3)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.bias', 'pre_classifier.we

In [14]:
args = TrainingArguments(
    f"distilbert-finetuned-mnli",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)



In [15]:
metric = evaluate.load("glue", "mnli")

In [16]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)


In [17]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation_matched"],
    compute_metrics=compute_metrics,
    tokenizer = tokenizer
)


In [18]:
trainer.evaluate()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 trainer.evaluate()                                                                           │
│   2                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:3053 in evaluate                 │
│                                                                                                  │
│   3050 │   │   start_time = time.time()                                                          │
│   3051 │   │                                                                                     │
│   3052 │   │   eval_loop = self.prediction_loop if self.args.use_legacy_prediction_loop else se  │
│ ❱ 3053 │   │   output = eval_loop(                                                               │
│   3054 │   │   │   eval_dataloader,                                                              │
│   3055 │   │   │   description="Evaluation",                                                     │
│   3056 │   │   │   # No point gathering the predictions if there are no metrics, otherwise we d  │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:3353 in evaluation_loop          │
│                                                                                                  │
│   3350 │   │   │   │   │   EvalPrediction(predictions=all_preds, label_ids=all_labels, inputs=a  │
│   3351 │   │   │   │   )                                                                         │
│   3352 │   │   │   else:                                                                         │
│ ❱ 3353 │   │   │   │   metrics = self.compute_metrics(EvalPrediction(predictions=all_preds, lab  │
│   3354 │   │   else:                                                                             │
│   3355 │   │   │   metrics = {}                                                                  │
│   3356                                                                                           │
│                                                                                                  │
│ in compute_metrics:3                                                                             │
│                                                                                                  │
│   1 def compute_metrics(eval_pred):                                                              │
│   2 │   predictions, labels = eval_pred                                                          │
│ ❱ 3 │   predictions = np.argmax(predictions, axis=1)                                             │
│   4 │   return metric.compute(predictions=predictions, references=labels)                        │
│   5                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'np' is not defined

In [ ]:
for batch in trainer.get_eval_dataloader():
    break

batch = {k: v.to(device) for k, v in batch.items()}

with torch.no_grad():
    outputs = trainer.model(**batch)

In [ ]:
predictions = outputs.logits.cpu().numpy()
labels = batch["labels"].cpu().numpy()

compute_metrics((predictions, labels))

In [ ]:
#trainer.train()

In [ ]:
for batch in trainer.get_train_dataloader():
    break

outputs = trainer.model.cpu()(**batch)

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
batch = {k: v.to(device) for k, v in batch.items()}
print(device)
outputs = trainer.model.to(device)(**batch)
outputs

In [ ]:
trainer.train_dataset[0]

In [ ]:
tokenizer.decode(trainer.train_dataset[0]['input_ids'])

In [ ]:
trainer.train_dataset[0].keys()

In [ ]:
type(trainer.model)

In [ ]:
print(trainer.train_dataset[0]["attention_mask"])

In [ ]:
len(trainer.train_dataset[0]["attention_mask"]) == len(trainer.train_dataset[0]["input_ids"])

In [ ]:
trainer.train_dataset[0]["label"]

In [ ]:
trainer.train_dataset.features["label"]

In [ ]:
trainer.train_dataset.features["label"].names[0]

In [ ]:
trainer.train_dataset.features["label"].names

In [ ]:
for batch in trainer.get_train_dataloader():
    break

In [ ]:
data_collator = trainer.get_train_dataloader().collate_fn
data_collator

In [ ]:
data_collator = trainer.get_train_dataloader().collate_fn
actual_train_set = trainer._remove_unused_columns(trainer.train_dataset)
batch = data_collator([actual_train_set[i] for i in range(4)])
batch

In [ ]:
trainer.model.config.num_labels